# Cover Page 
### Student ID: 210003508
### Module Code: GG4257 
### Module Title: Urban Analytics: A Toolkit for Sustainable Urban Development
### Assignment: Lab Assignment No 2 - Networks, Geodemographics and Spatial Microsimulation.
### Degree Programme: Geography 
### Deadline Date: 02.04.2025

In submitting this assignment, I hereby confirm that:

I have read the University's statement on Good Academic Practice; that the following work is my own work; and that significant academic debts and borrowings have been properly acknowledged and referenced.

# Introduction 

This section outlines how to replicate the code and access the required data. All data will be available in a **OneDrive folder**, with the code provided in a dedicated GitHub repository.

This report documents the work conducted for **Lab Assignment 2**, covering challenges from **Labs 5, 6 and 7**. Each lab section includes problem descriptions, methods, and results. Code is supplemented with comments and markdown explanations, with screenshots of outputs (e.g., maps and graphs) included to ensure clarity and help replication. To meet GitHub size limits, certain output cells have been cleared from the notebook. All data paths in the code assume files are stored in a folder named "Data". 

#### GitHub Username: Ejarrett
#### GitHub Repository: [UA_Lab_Assignment_2](https://github.com/ejarrettt/UA_Lab_Assignment_2)
#### [OneDrive Folder](https://1drv.ms/f/c/46775ff05561cd60/EtpKCVLnUXtMupoCXG5G86QB-GfpojgYMR0f4hgJI-Fclg?e=bvGWOb)

To replicate this report:
1. Clone the GitHub repository.
2. Download the datasets from the [OneDrive Folder](https://1drv.ms/f/c/46775ff05561cd60/EtpKCVLnUXtMupoCXG5G86QB-GfpojgYMR0f4hgJI-Fclg?e=bvGWOb)
3. Unzip the folder and copy the "Data" folder to the same place you have stored the notebook. 
4. Run this notebook. 

# Lab No 5: Introduction to Networks (2 Challenges)

In [ ]:
# Install dependencies 
! pip install -r requirements.txt

## Challenge 1:

It's time for you to apply everything you learned by analyzing a case study of FourSquare social Network. (Foursquare is a location-based online social network. The dataset contains a list of all of the user-to-user links)

Datasource: @inproceedings{gao2012exploring,
     title={Exploring social-historical ties on location-based social networks},
     author={Gao, Huiji and Tang, Jiliang and Liu, Huan},
     booktitle={Proceedings of the 6th International AAAI Conference on Weblogs and Social Media},
     year={2012}
}

- **Data**: `FS.csv` (avaliable in Moodle)

1. Read the FS network dataset.
2. Describe using the basic functions of the graph's size. Explore nodes and edges. Provide how many nodes and edges are present in the network.
3. The dataset generates a graph of 639.014 nodes, so it is massive and you won't see anything meaningful if you try to plot it. So you need to create a subset using the **degree centrality** to find out find the top 4 of the most important nodes, and use them to create a subset of the original network. 
4. Extract the degree centrality values and convert them into a list. Then, plot a histogram to visualize the distribution of node degrees in the original network.
5. Create a plot for the subset created.
6. Now calculate another relevant measure of the network -- **betweenness centrality**. Plot the betweenness centrality distribution of the subset you created. Tip: Same steps from the previous step, but use `nx.betweenness_centrality()`
7. Plot the Matrix, Arc and Circos from the subset.

## Challenge 2: 

This challenge is about OSMnx. You will explore and analyze a city's street network using the OSMnx Python library.

1. Use OSMnx to download the street network of a city of your choice. You can specify the city name, BBox or a Dict.
2. Calculate basic statistics for the street network, such as the number of nodes, edges, average node degree, etc.
3. Use OSMnx to plot the street network. Customize the plot to make it visually appealing, including node size, edge color. See the potential options here: https://osmnx.readthedocs.io/en/stable/user-reference.html#module-osmnx.plot
4. Utilize the routing capabilities of OSMnx to find the shortest path between two points in the street network. Plot the route on top of the street network.
5. Calculate the centrality measures (e.g., degree centrality and betweenness_centrality) for nodes in the street network.
6. Create the figure-groud from the selected city
7. Create interactive maps to plot nodes, edges, nodes+edges and one of the centrality measures.
8. Export the street network to a GeoPackage (.gpkg) file. Ensure that the exported file contains both node and edge attributes. Demonstrate that the new GeoPackage can be used and read in Python using any of the libraries we have seen in the class to create a simple and interactive map.
9. Finally, use OSMnx to extract other urban elements (e.g., buildings, parks) and plot them.

# Lab No 6: Geodemographics (1 Challenge)

## Challenge 1: Geodemographic Classification

In this challenge, you will replicate the process of creating a geodemographic classification using the k-means clustering algorithm. Please select any city in the UK except London, Liverpool, or Glasgow. The main goal is to generate a meaningful and informative classification that captures the diversity of areas in your dataset using the census data ( For England, you can try to use the 2021 or 2011 census, and for Scotland, you need to use the 2011 census data) 

1. Define the main goal for the geodemographic classification (marketing, retail and service planning). 
2. Look for census data from the selected city for which you would like to generate the geodemographic classification.
3. The census data at the Output Area OA level. Select multiple topics of at least four topics (socio-demographics, economics, health, and so on). Describe your topic selection accordingly based on the goal of your geodemographic classification. For example, if your geodemographics are related to marketing, Economic variables might be the appropriate selection. 
4. Identify the variables that will be crucial for effectively segmenting neighbourhoods. Evaluate how this choice may impact the classification results, including a DEA analysis.
5. Prepare, adjust or clean the dataset addressing any missing values or outliers that could distort the clustering results.
6. Include standardisation between areas and variables. Make an appropriate analysis and adjust the variable selection accordingly for any multicollinearity.
7. Utilize the k-means clustering algorithm to create a classification based on the selected variables.
8. Define the optimum number of clusters (i.e., using the Elblow method). Experiment with different values of k.
9. Evaluate your cluster groups (e.g., using PCA) and interpret your cluster centres. Describe your results and repeat the process to adjust the variable selection and cluster groups to provide more meaningful results for your geodemographic goal. Interpret the characteristics of each cluster. What demographic patterns or similarities are prevalent within each group?
10. Map the final cluster groups
11. Finish the analysis by naming the final clusters and plotting a final map that includes the census values and the provided names.
12. Finally, acknowledge the subjective nature of classification and make analytical decisions to produce an optimum classification for your specific purpose. Reflect on the challenges and insights gained during the classification process. Ensure you document your analytical decisions and the rationale behind any important decision. Once your geodemographics are constructed, describe the potential use cases for the geodemographic classification you have built based on your initial goal.

# Lab No 7: Spatial Microsimulation (3 Challenges)

## Challenge 1: 

## Challenge 2: 

## Challenge 3: 

# Final Remarks (limitations, barriers, and any additional comments)